- 페이지 소스보기 했을 때 원하는 데이터가 없다.
- 화면에는 원하는 데이터가 있다.
- 크롬의 개발자 도구를 띄워주고 network 탭을 눌러준다.
- 페이지에 변화를 주고 network 탭에 떠있는 주소 정보에 xhr이라고 되어 있는지 확인한다.
- xhr 이라고 되어 있으면 비동기통신이다.
- xhr 이라고 되어 있는 곳의 name이 요청 주소가 되며 name 부분을 더블클릭해서 띄워본다.

In [7]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os
from IPython.display import clear_output

In [8]:
# 요청 함수
def getSource(site) :
    # 해더 정보 셋팅
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    # 요청한다.
    response = requests.get(site, headers=header_info)
    return response.text

In [9]:
# 한 페이지의 데이터를 가져오는 함수
def getData(source) :
    # bs4 객체를 생성한다.
    soup = bs4.BeautifulSoup(source)
    
    # 경기 덩어리들을 가져온다.
    li_list = soup.select('#todaySchedule > li')
    
    # 데이터를 담을 딕셔너리
    data_dict = {
        '팀1이름' : [],
        '팀2이름' : [],
        '팀1점수' : [],
        '팀2점수' : [],
    }
    
    for li_tag in li_list :
        # print(li_tag)
        
        # class 속성이 있는가..
        if 'class' in li_tag.attrs :
            # class 속성들을 가지고 온다.
            a1 = li_tag.attrs['class']
            # end 라는 class 속성이 있는가..
            if 'end' in a1 :
                # print(a1)
                
                # 팀1 이름
                a10 = li_tag.select_one('div.vs_lft > p > strong')
                data1 = getText(a10)
                
                # 팀2 이름
                a20 = li_tag.select_one('div.vs_rgt > p > strong')
                data2 = getText(a20)
                
                # 팀1 점수
                a30 = li_tag.select_one('div.vs_lft > strong')
                data3 = getText(a30)
                
                # 팀2 점수
                a40 = li_tag.select_one('div.vs_rgt > strong')
                data4 = getText(a40)
                
                # print(data1)
                # print(data2)
                # print(data3)
                # print(data4)
                # print('----------------------------------------')
                
                
                data_dict['팀1이름'].append(data1)
                data_dict['팀2이름'].append(data2)
                data_dict['팀1점수'].append(data3)
                data_dict['팀2점수'].append(data4)
        
    df1 = pd.DataFrame(data_dict)
    
    if os.path.exists('./06_data.csv') == False :
        df1.to_csv('./06_data.csv', encoding='utf-8-sig', index=False)
    else :
        df1.to_csv('./06_data.csv', encoding='utf-8-sig', index=False, header=None, mode='a')

In [10]:
# 전달된 태그로 부터 문자열 데이터를 추출해 반환하는 함수
def getText(tag):
    # 태그가 없다면
    if tag == None :
        return np.nan
    # 문자열을 가져온다.
    data = tag.text.strip()
    # 길이가 0 이라면...
    if len(data) == 0 :
        return np.nan
    return data

In [11]:
# 다음에 수집할 날짜를 가져온다.
def getNextDate(source, today) :
    
    soup = bs4.BeautifulSoup(source)
    
    # 날짜 덩어리들을 가져온다.
    li_list = soup.select('body > div > ul > li')
    if today == '20080329':
        next_tag = li_list[2]
    elif today == '20080330':
        next_tag = li_list[3]
    elif today == '20080401':
        next_tag = li_list[4]
    else :
        next_tag = li_list[5]
        
    # print(next_tag)
    
    # next_tag안의 a태그를 가져온다.
    a_tag = next_tag.select_one('a')
    # print(a_tag)
    # onclick 속성을 가져온다.
    onclick = a_tag.attrs['onclick']
    # print(onclick)
    
    # 작은 따옴표를 기준으로 잘라낸다.
    a100 = onclick.split("'")
    next_day = a100[1]
    # print(next_day)
    return next_day

In [12]:
today = '20080329'
while True :
    time.sleep(1)
    clear_output(wait=True)
    
    print(f'{today} 수집중...')
    
    site = f'https://sports.news.naver.com/schedule/scoreBoard?date={today}&category=kbo&direction=before'

    # 요청한다.
    source = getSource(site)
    # print(source)

    # 한 페이지의 데이터를 가져온다.
    getData(source)

    # 다름 날짜를 가져온다.
    today = getNextDate(source, today)
    # print(today)
    
    # 가져온 다음 날짜가 2022로 시작하면 while문을 중단한다.
    if today.startswith('2022'):
        print('수집완료')
        break

20211118 수집중...
수집완료
